In [1]:
!pip install num2words

     |████████████████████████████████| 102kB 2.3MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, words, wordnet
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from keras.preprocessing.text import Tokenizer
from num2words import num2words 
from keras.preprocessing.sequence import pad_sequences



# from replacers import RegexpReplacer, RepeatReplacer


from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm

Using TensorFlow backend.


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id': '12jvCA-Ik2Yvy_60-kKJnBWadRxIuVLq'}) 
downloaded.GetContentFile('glove.42B.300d.txt')

In [ ]:
downloaded = drive.CreateFile({'id':"1PTjUlqfdULPkXLoM1Xw7jhXzkuIHQybu"}) 
downloaded.GetContentFile('hinglish_stemmer.py')

In [ ]:
congress_train_data = pd.read_csv('drive/My Drive/TextSimilarity/CONGRESS.csv')
congress_train_data["about_bjp"] = 0
congress_train_data["about_congress"] = 1
congress_train_data.head()

,commentText,Label,about_bjp,about_congress
0,Modi ji super,0,0,1
1,"You were fool, you are fool and you will be fool",0,0,1
2,Rahul fool,0,0,1
3,Rahul is best as PM,1,0,1
4,Rahul Gandhi theif he,0,0,1


In [18]:
train_data = pd.read_csv('drive/My Drive/Twitter-SentimentAnalysis/hate_speech.tsv',
                         sep='\t', 
                         names = ['SentimentText', "Sentiment"], 
                         header = None)

train_data.head()

,SentimentText,Sentiment
0,Knowing ki Vikas kitna samjhata hai Priyanka a...,no
1,I am Muhajir .. Aur mere lye sab se Pehly Paki...,no
2,Doctor sab sahi me ke PhD (in hate politics) ...,no
3,Poore Desh me Patel OBC me aate Hain sirf gujr...,no
4,Sarkar banne ke bad Hindu hit me ek bhi faisla...,yes


In [19]:
train_data['Sentiment']  = train_data['Sentiment'].apply(lambda x: 1 if x == "yes" else 0)

In [20]:
train_data.head()

,SentimentText,Sentiment
0,Knowing ki Vikas kitna samjhata hai Priyanka a...,0
1,I am Muhajir .. Aur mere lye sab se Pehly Paki...,0
2,Doctor sab sahi me ke PhD (in hate politics) ...,0
3,Poore Desh me Patel OBC me aate Hain sirf gujr...,0
4,Sarkar banne ke bad Hindu hit me ek bhi faisla...,1


In [21]:
train_data.shape

(4579, 2)

In [23]:
train_data.dropna(inplace=True)

In [24]:
train_data.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            4569, 4570, 4571, 4572, 4573, 4574, 4575, 4576, 4577, 4578],
           dtype='int64', length=4578)

In [25]:
train_data = train_data.sample(frac=1).reset_index(drop=True)

In [26]:
train_data.SentimentText[2]

'Wat is this nonsense jo witnesses mare gaye unka zimmevar kon h koi bhi lady khamakha thori khaegi ki mera rape hua #CleanChitToAsaramBapuji'

In [27]:
#Checking Tweets
rand_indexs = np.random.randint(1,len(train_data),10).tolist()
train_data["SentimentText"][rand_indexs]

7       Delhi gang rape K kamino ko Abhi tk jinda rkha...
4425    I hate bre bre nails wali kuriyan ... susti le...
760                            sir ji pi rakhi hai kya ??
4485                 tak payah tak payah. I hate suprise.
3780    I hate this "ngalusa mina" nonsense. Inobuphum...
1465    Niazi HaramKhor Pakistan ko Terrorism mai dhka...
4212    Rajasthan mein 1 nabalig ladaki ke sath rape h...
2482    Modi se phoocho....Haryana jal Raha hai...Modi...
3458    Agar tum use support nahi karoge to tumhari no...
1665    Achcha choddo yaar. Tum log Jeeton aur hazaaro...
Name: SentimentText, dtype: object

In [28]:
tweets_text = train_data.SentimentText.str.cat()
emos = set(re.findall(r" ([xX:;][-']?.) ",tweets_text))
emos_count = []
for emo in emos:
    emos_count.append((tweets_text.count(emo), emo))
sorted(emos_count,reverse=True)[:10]

[(40, ': '),
 (10, ':)'),
 (6, ':P'),
 (6, ':('),
 (4, ':D'),
 (3, ':p'),
 (2, ';)'),
 (1, ';p')]

In [29]:
HAPPY_EMO = r" ([xX;:]-?[dD)]|:-?[\)]|[;:][pP]|<3)"
SAD_EMO = r" (:'?[/|\(]) "
print("Happy emoticons:", set(re.findall(HAPPY_EMO, tweets_text)))
print("Sad emoticons:", set(re.findall(SAD_EMO, tweets_text)))

Happy emoticons: {':p', ';p', ':)', ';)', ':D', '<3', ':P'}
Sad emoticons: {':('}


In [30]:
nltk.download('punkt')
nltk.download('words')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [31]:
def most_used_words(text):
    tokens = word_tokenize(text)
    frequency_dist = nltk.FreqDist(tokens)
    print("There is %d different words" % len(set(tokens)))
    return sorted(frequency_dist,key=frequency_dist.__getitem__, reverse=True)

In [32]:
most_used_words(train_data.SentimentText.str.cat())[:10]

There is 20176 different words


['hai', 'rape', 'ki', ',', 'ko', 'ka', '.', 'ke', 'hate', 'se']

In [33]:
fobj = open("drive/My Drive/Twitter-SentimentAnalysis/stop_hinglish.txt")
stopwords_hin = fobj.read().split()

stopwords_eng = stopwords.words('english')

stopwords = stopwords_eng + stopwords_hin

In [35]:
stopwords[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [ ]:
import hinglish_stemmer

def remove_stop(tokenized):

  global nlp

  #Removing Stopwords (hinglish + english)
  new_text = []
  for token in tokenized:
    if (token in stopwords_hin) or (token in stopwords_eng):
      continue
    else:
      new_text.append(token)

  return " ".join(new_text)

def lemmatizing_preprocess(text):

  lemmatizer = WordNetLemmatizer()
  tokenized = word_tokenize(text)
  hinglish_stem = hinglish_stemmer.Stemmer()

  lemmatized = []

  for t in tokenized:
    if t in words.words():
      lemmatized.append(lemmatizer.lemmatize(t))
    else:
      lemmatized.append(hinglish_stem.stemWord(t))

  return " ".join(lemmatized)



In [36]:
replacement_patterns = [
(r'won\'t', 'will not'),
(r'can\'t', 'can not'),
(r'i\'m', 'i am'),
(r'ain\'t', 'is not'),
(r'(\w+)\'ll', '\g<1> will'),
(r'(\w+)n\'t', '\g<1> not'),
(r'(\w+)\'ve', '\g<1> have'),
(r'(\w+)\'s', '\g<1> is'),
(r'(\w+)\'re', '\g<1> are'),
(r'(\w+)\'d', '\g<1> would')
]

class RegexpReplacer(object):

   def __init__(self, patterns=replacement_patterns):
     self.patterns = [(re.compile(regex), repl) for (regex, repl) in patterns]

   def replace(self, text):
     s = text
     for (pattern, repl) in self.patterns:
       s = re.sub(pattern, repl, s)

     return s


class RepeatReplacer(object):
    def __init__(self):
        self.repeat_regexp = re.compile(r'(\w*)(\w)\2(\w*)')
        self.repl = r'\1\2\3'
    def replace(self, word):
        if wordnet.synsets(word):
            return word
        repl_word = self.repeat_regexp.sub(self.repl, word)
        if repl_word != word:
            return self.replace(repl_word)
        else:
            return repl_word



In [37]:
import string


def preprocessing(text):
  pattern='https?:\S+|[^a-zA-Z0-9]+'
  text=re.sub(pattern,' ',text.lower()).strip()
  tokens=[]
  for token in text.split():
    try:
      token=num2words(token)
    except:
      pass
    # if token not in stop_words:
    # tokens.append(lemmatizer.lemmatize(token))
    tokens.append(token)
    # tokens.append(stemmer.stem(token))
  return " ".join(tokens)


train_data['SentimentText'] = train_data['SentimentText'].str.replace(HAPPY_EMO, " happy ")
train_data['SentimentText'] = train_data['SentimentText'].str.replace(SAD_EMO, " sad ")

train_data['SentimentText'] = train_data['SentimentText'].apply(preprocessing)



train_data.head()

,SentimentText,Sentiment
0,apney bachon ki hiffazet kero ye bhi ibbadet h...,0
1,aur men ne kab kaha k rape aur killing karo me...,0
2,wat is this nonsense jo witnesses mare gaye un...,1
3,yaad rakhna aise hi kisine peshawar school ke ...,0
4,very true but ishake liye bhi khoon pashine ki...,0


In [38]:
train_data.to_csv("drive/My Drive/Twitter-SentimentAnalysis/cleanedData.csv")

In [ ]:
# train_data = pd.read_csv('drive/My Drive/Twitter-SentimentAnalysis/cleanedData.csv')

In [ ]:
# train_data['SentimentText'] = train_data['SentimentText'].apply(remove_stop)

In [39]:
train_data.shape

(4578, 2)

In [40]:
train_x, valid_x, train_y, valid_y = train_test_split(train_data[['SentimentText']], train_data['Sentiment'], test_size=0.15,random_state=42)

In [41]:
np.quantile([len(x) for x in list(train_data.SentimentText)], [0.90])

array([152.])

In [42]:
documents = [t.split() for t in train_data.SentimentText]

## Character Embedding

In [ ]:
import keras.backend as K
import keras.initializers

In [ ]:
# Tokenizer
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(documents)


alphabet="abcdefghijklmnopqrstuvwxyz0123456789 ,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"

char_dict = {}
for i, char in enumerate(alphabet):
    char_dict[char] = i + 1
    
# Use char_dict to replace the tk.word_index
tk.word_index = char_dict.copy() 
# Add 'UNK' to the vocabulary 
tk.word_index[tk.oov_token] = max(char_dict.values()) + 1


In [ ]:
sentiment_text = pad_sequences(tk.texts_to_sequences(train_data['SentimentText']), maxlen=550, padding="post")
sentiment_text.shape

(3950, 550)

In [ ]:
inputs_chars = len(tk.word_counts) + 1
inputs_chars

5357

In [ ]:
vocab_size = len(tk.word_index) 
vocab_size

70

In [ ]:
char_embeddings = [] #(71, 70)
char_embeddings.append(np.zeros(vocab_size)) # first row is pad

for char, i in tk.word_index.items(): # from index 1 to 70
    onehot = np.zeros(vocab_size)
    onehot[i-1] = 1
    char_embeddings.append(onehot)
char_embeddings = np.array(char_embeddings)

In [ ]:
y_train

array([0, 0, 1, ..., 0, 1, 0])

## Word Embeddings

In [43]:
import gensim
w2v_model=gensim.models.word2vec.Word2Vec(size=300,window=5,min_count=10,workers=8)
w2v_model.build_vocab(documents)

In [44]:
words = w2v_model.wv.vocab.keys()
vocab_size = len(words)
print("Vocab size", vocab_size)

Vocab size 1103


In [45]:
w2v_model.train(documents,total_examples=len(documents),epochs=32)

(1578041, 2960800)

In [46]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(documents)

In [47]:
sentiment_text = pad_sequences(tokenizer.texts_to_sequences(train_data.SentimentText),maxlen=150)

In [48]:
vocab_size=len(tokenizer.word_index)+1

embedding_matrix=np.zeros((vocab_size,300))

for word,i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i]=w2v_model.wv[word]

## Model

In [80]:
testing_labels = pd.get_dummies(train_data['Sentiment'])
testing_labels[:10]

,0,1
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0
5,1,0
6,1,0
7,1,0
8,1,0
9,1,0


In [81]:
from keras.models import Model, Input
from keras.layers import GRU, LSTM, Bidirectional, Conv1D, GlobalMaxPool1D, Embedding, Dense, MaxPooling1D, GlobalMaxPooling1D, Concatenate

In [82]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sentiment_text, np.array(testing_labels), test_size=0.2, random_state=0)

In [83]:
from tensorflow.keras import backend as K
import tensorflow as tf

def f1_m(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    # tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [84]:
embedding_layer = Embedding(vocab_size, 300, trainable=True, weights=[embedding_matrix])
embedding_layer_input = Input(shape=(150,))
embedding_layer_x = embedding_layer(embedding_layer_input)

In [100]:
gru = Bidirectional(GRU(100, return_sequences=True))
gru_ouputs = gru(embedding_layer_x)
after_gru = GlobalMaxPooling1D()(gru_ouputs)

In [101]:

output_category = Dense(2, activation='sigmoid')(after_gru)


In [102]:
model = Model(embedding_layer_input, output_category)

In [104]:
from tensorflow.keras.optimizers import Adam,SGD,Adadelta

opt=Adadelta(learning_rate=1e-4)

model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy', f1_m])
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 150)               0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 150, 300)          4266300   
_________________________________________________________________
bidirectional_6 (Bidirection (None, 150, 200)          240600    
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 200)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 2)                 402       
Total params: 4,507,302
Trainable params: 4,507,302
Non-trainable params: 0
_________________________________________________________________


In [105]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

callbacks = [
            #  EarlyStopping(monitor='val_loss', patience=8, verbose=1, min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, cooldown=0, min_lr=1e-7, verbose=1)
]

model.fit(X_train, y_train, batch_size=4096, epochs=40, validation_split=0.2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2929 samples, validate on 733 samples
Epoch 1/40
2929/2929 [==============================] - 3s 917us/step - loss: 0.7460 - accuracy: 0.3911 - f1_m: 0.3713 - val_loss: 0.6791 - val_accuracy: 0.5962 - val_f1_m: 0.6217
Epoch 2/40
2929/2929 [==============================] - 2s 570us/step - loss: 0.6798 - accuracy: 0.5888 - f1_m: 0.6169 - val_loss: 0.6404 - val_accuracy: 0.6774 - val_f1_m: 0.5202
Epoch 3/40
2929/2929 [==============================] - 2s 629us/step - loss: 0.6484 - accuracy: 0.6646 - f1_m: 0.5225 - val_loss: 0.6269 - val_accuracy: 0.6726 - val_f1_m: 0.4636
Epoch 4/40
2929/2929 [==============================] - 2s 583us/step - loss: 0.6411 - accuracy: 0.6559 - f1_m: 0.4548 - val_loss: 0.6245 - val_accuracy: 0.6733 - val_f1_m: 0.4546
Epoch 5/40
2929/2929 [==============================] - 2s 593us/step - loss: 0.6425 - accuracy: 0.6567 - f1_m: 0.4501 - val_loss: 0.6226 - val_accuracy: 0.6760 - val_f1_m: 0.4679
Epoch 6/40
2929/2929 [==============================]

In [106]:
model.evaluate(X_test,y_test)

916/916 [==============================] - 2s 2ms/step


[0.5514121472054694, 0.7281659245491028, 0.6604800224304199]

## Using Count Vectorizer

In [ ]:
from nltk.tokenize import word_tokenize

def tokenization(tweet):
    text = []
    for token in word_tokenize(tweet):
        if token in stopwords:
            continue
        else:
            text.append(token)
    return word_tokenize(tweet)

In [ ]:
count_vect = CountVectorizer(analyzer='word', tokenizer=tokenization)
count_vect.fit(train_data['SentimentText'])

xtrain_count =  count_vect.transform(train_x['SentimentText'])
xvalid_count =  count_vect.transform(valid_x['SentimentText'])

feature_names = count_vect.get_feature_names()
dense = xtrain_count.todense()
denselist = dense.tolist()
xtrain_count = pd.DataFrame(denselist, columns=feature_names)
xtrain_count.reset_index()

dense = xvalid_count.todense()
denselist = dense.tolist()
xvalid_count = pd.DataFrame(denselist, columns=feature_names)
xvalid_count.reset_index()

train_x[['about_bjp', 'about_congress']].reset_index().drop(columns=['index'], inplace=True)
valid_x[['about_bjp', 'about_congress']].reset_index().drop(columns=['index'], inplace=True)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [ ]:
xtrain_count

,",",10days,10m,15lakh,1k,1m,1st,2019all,24x7,2g,2k19,2nd,2points,300plus,30cr,3lacs,3rd,420k,4th,4years,56inch,5decades,60years,70s,70thousand,8th,9th,a,aa,aaaae,aab,aadmi,aage,aagya,aaj,aak,aakhri,aam,aana,aandhi,...,yogendra,yogi,yogjji,yojan,yoooo,york,you,young,younger,youngest,youngsters,your,yourself,youself,youth,youths,youu,yr,yrr,yrs,yt,yuganda,zada,zameen,zaroor,zee,zeenews,zehadis,zero,zeros,zi,zimdabad,zinda,zindabaad,zindabad,zindagi,zindbad,zindhabad,zulm,zyada
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3352,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3353,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3354,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3355,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
xtrain_count = np.hstack([xtrain_count, train_x[['about_bjp', 'about_congress']]])
xvalid_count = np.hstack([xvalid_count, valid_x[['about_bjp', 'about_congress']]])

In [ ]:
xtrain_count

,",",10days,10m,15lakh,1k,1m,1st,2019all,24x7,2g,2k19,2nd,2points,300plus,30cr,3lacs,3rd,420k,4th,4years,56inch,5decades,60years,70s,70thousand,8th,9th,a,aa,aaaae,aab,aadmi,aage,aagya,aaj,aak,aakhri,aam,aana,aandhi,...,yogendra,yogi,yogjji,yojan,yoooo,york,you,young,younger,youngest,youngsters,your,yourself,youself,youth,youths,youu,yr,yrr,yrs,yt,yuganda,zada,zameen,zaroor,zee,zeenews,zehadis,zero,zeros,zi,zimdabad,zinda,zindabaad,zindabad,zindagi,zindbad,zindhabad,zulm,zyada
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3352,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3353,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3354,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3355,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
classifier = naive_bayes.MultinomialNB()
# classifier = svm.SVC()
# classifier = xgboost.XGBClassifier()
classifier.fit(xtrain_count, train_y)
predictions = classifier.predict(xvalid_count)
print(predictions.shape)
metrics.accuracy_score(predictions, valid_y)

(593,)


0.5396290050590219

In [ ]:
predictions

array([1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0,

In [ ]:
class Vectorizer(BaseEstimator,TransformerMixin):
    def __init__(self, max_features = 2000, ngram_range = (1,3), tokenizer = None):
      self.max_features, self.ngram_range, self.tokenizer = max_features, ngram_range, tokenizer

    def transform(self, X, y = None):

      vectorizer = TfidfVectorizer(max_features = self.max_features, 
                                   ngram_range = self.ngram_range,
                                   tokenizer = self.tokenizer)
      
      vectors = vectorizer.fit_transform(X['SentimentText'].values.astype('U'))
      feature_names = vectorizer.get_feature_names()
      dense = vectors.todense()
      denselist = dense.tolist()
      df_vect = pd.DataFrame(denselist, columns=feature_names)
      df_vect.reset_index()

      df_vect = pd.concat([df_vect, X[['about_bjp', 'about_congress']]], axis = 1)

      return df_vect

    def fit(self, X, y=None):
      return self
      



In [ ]:
tweets = train_data[['SentimentText', 'about_bjp', 'about_congress']]

vect = Vectorizer(tokenizer = word_tokenize)

X = vect.fit_transform(tweets)
X

,1,a,a a,a big,a bjp,a chance,a chance to,a day,a educated,a god,a great,a man,a next,a next pm,a pm,a prime,a prime minister,a real,a single,a the,a true,a usual,a very,ab,able,about,about the,above,abt,acept,aceptable,acha,ache,acording,acording to,act,acting,action,actions,actualy,...,ye,year,years,yeh,yes,yet,you,you are,you can,you congres,you guys,you have,you know,you modi,you modi ji,you rahul,you rahul gang,you want,you wil,young,young leader,youngsters,your,youth,yrs,zero,zero crores,zero one,zero one four,zero one nine,zero seats,zero two,zero year,zero years,zero yrs,zindabad,zulm,😀,about_bjp,about_congress
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.137179,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.084064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3945,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3946,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3947,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3948,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()

train_data = train_data.reindex(np.random.permutation(train_data.index))
train_data.reset_index()

tweets = X
# train_data[['SentimentText', 'about_bjp', 'about_congress']]
sentiments = train_data[['Sentiment']]


def tfidf_tokenizer(text):
  return word_tokenize(text)

clf = SVC(probability=True, C=10, gamma=0.1, kernel='rbf')
vect = Vectorizer(tokenizer = word_tokenize)


grid_search_pipeline = Pipeline([
    # ('vectorizer', vect),
    ('SVC', clf)
])


print(tweets.shape)

learn_data, test_data, sentiments_learning, sentiments_test = train_test_split(tweets, sentiments, test_size=0.3)

vectorized_data = grid_search_pipeline.fit(learn_data, sentiments_learning)


# params = [
#     {
#         # 'vectorizer__tokenizer': [tfidf_tokenizer],
#         'vectorizer__max_features': [1000, 2000],
#         'vectorizer__ngram_range': [(1,3)],
#         'SVC__C': [0.1, 1, 10],  
#         'SVC__gamma': [0.1, 0.01], 
#         'SVC__kernel': ['rbf']
#     },
# ]



# grid_search = GridSearchCV(grid_search_pipeline, params, cv=5, scoring='f1')
# grid_search.fit(tweets, sentiments)
# print(grid_search.best_params_)

(3950, 2002)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
k = grid_search_pipeline.predict(learn_data)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

accuracy_score(sentiments_learning, k)

0.8311030741410488

In [ ]:
f1_score(sentiments_test, k, average="micro")

In [ ]:
k

array([ 0,  0, -1, ...,  0,  0,  1])

In [ ]:
sentiments_test

304     0
3106   -1
818    -1
6316    0
5395    0
       ..
7565    0
5213    1
3157    1
3746   -1
3556    1
Name: Sentiment, Length: 3843, dtype: int64